In [ ]:
from bimodal_dist import bimodal, bimodal_rev, generate_dist, HistDist, ContinuousDist
from free_energy import bennet_delta_F, class_meta_f, get_proportion_error, variance_plot, tcft_correction

import sys
tf_path = '/Users/kray/Documents/UCD/Crutchfield_Group/thermoflow/'
sys.path.append(tf_path)
from thermoflow import free_energy as tffe

In [ ]:
from bimodal_dist import dist_from_piecewise_list, rdist_from_piecesise_dist

In [ ]:
base_shape = [-1,-.5,.5,1]

xp = [item+4 for item in base_shape] + [item+7 for item in base_shape]

fp = [0, .4,.4,0, 0,1,1,0]

dist = dist_from_piecewise_list(xp, fp)
r_dist = rdist_from_piecesise_dist(dist)


F = -np.log(dist.negexp_avg())
FR = -np.log(r_dist.negexp_avg())
print(F, FR)

LL, UL = [-5,5]

In [ ]:
#x = np.linspace(-(4.5+F),4.5+F, 3000)
X = np.max(np.abs(xp))+1
x = np.linspace(-X,X, 3000)
fig, ax = plt.subplots(1,3, figsize=(15,4), sharex=True)
ax[0].plot(x, [dist.pdf(xv) for xv in x], label='$F(x)$')
ax[0].plot(x, [r_dist.pdf(xv) for xv in x], label='$R(x)$')

ax[1].plot(x, [dist.pdf(xv) for xv in x], label='$F(x)$')
ax[1].plot(-x, [r_dist.pdf(xv) for xv in x], label='$R(-x)$', linestyle='--', zorder=100)

ax[2].plot(x, [dist.pdf(xv) for xv in x], label='$F(x)$')
ax[2].plot(-x, [r_dist.pdf(xv) for xv in x], label='$R(-x)$', linestyle='--', zorder=100)





ax[0].axvspan(LL,UL, color='tab:blue', alpha=.3, label='$C$')
ax[0].axvspan(-UL,-LL, color='tab:orange', alpha=.3, label='$C^{R}$')

ax[1].axvspan(LL,UL, color='tab:grey', alpha=.3, label='$C$ and $C^{R}$')
ax[2].axvspan(LL,UL, color='tab:grey', alpha=.3, label='$C$ and $C^{R}$')

for l in [F]:
    for a in ax.ravel():
        a.axvline(l, alpha=1, c='k')
    

#ax[0].set_xlim(-3,3)
ax[2].set_yscale('log')
#ax[3].set_yscale('log')

for a in ax.ravel():
    a.legend()

In [ ]:
#fig.savefig('dist_and_classPoverR7p5.png')

In [ ]:
bins = np.linspace(*dist.lims,5_000)
rbins = -bins[::-1]

hdist = HistDist(dist.pdf, bins)
r_hdist = HistDist(r_dist.pdf,rbins)

In [ ]:
test_N = 25_000
s = hdist.rejection_sample(test_N)
sr = r_hdist.rejection_sample(test_N)

In [ ]:

b1 = (s>LL) * (s<UL)
b2 = (sr<-LL) * (sr>-UL)
print(sum(b1)/len(b1), sum(b2)/len(b2))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,8))
for a in ax.ravel():
    a.hist(s, bins=50, density=True, alpha=.6);
    a.hist(sr, bins=50, density=True, alpha=.6);
    a.plot(x, [dist.pdf(xv) for xv in x])
    a.plot(x, [r_dist.pdf(xv) for xv in x])

ax[1].set_yscale('log')


In [ ]:
BAR = []
JAR = []
JAR2 = []
GBAR = []
MBAR = []
GBARL = []
GJAR = []
GJARGAUSS = []
GBAYES = []

corr=[]
JARTC = []
JARTCC = []
BARTC = []
BARTCC = []

N=15

trials = 50
for i in range(trials):
    print(f'{(i+1)/trials:.2f}', end='\r')
    s = hdist.rejection_sample(2*N)
    sr = r_hdist.rejection_sample(N)
    
    b1 = (s[:N]>LL) * (s[:N]<UL)
    b2 = (sr[:N]<-LL) * (sr[:N]>-UL)
    for b in [b1,b2]:
        b = np.append(b,[True, False])

    if sum(b1) == 0 or sum(b2) == 0:
        print('zero class')
        continue
    
    corr.append([(sum(b1)+1)/(len(b1)+2), (sum(b2)+1)/(len(b2)+2)])


    #BAR.append( bennet_delta_F(s[:N], sr[:N]) )
    print('J1', end='\r')
    JAR.append( class_meta_f(s[:2*N]) )
    print('J2', end='\r')
    JAR2.append([tffe.free_energy_logmeanexp(s[:2*N]), 0])
    print('GG', end='\r')
    GJARGAUSS.append( [tffe.free_energy_logmeanexp_gaussian(s),0] )
    #print('GB', end='\r')
    #GBAYES.append(tffe.free_energy_bayesian(s[:N],sr[:N]))
    print('BR1', end='\r')
    GBAR.append( tffe.free_energy_bar(s[:N], sr[:N]) )
    print('BR2', end='\r')
    MBAR.append( tffe.free_energy_bar(s[:N], sr[:N],uncertainty_method='MBAR') )
    print('BR3', end='\r')
    GBARL.append( tffe.free_energy_bar(s[:N], sr[:N],uncertainty_method='Logistic') )
    print('TC', end='\r')
    JARTC.append(class_meta_f(s[:N][b1]))
    print('CC', end='\r')
    JARTCC.append(class_meta_f(s[:N][b1], probability_bools=[b1,b2]))
    print('BC', end='\r')
    BARTC.append(tffe.free_energy_bar(s[:N][b1], sr[:N][b2], uncertainty_method='MBAR'))
    print('TT', end='\r')
    tcft = tcft_correction(b1, b2)
    print('BB', end='\r')
    BARTCC.append( [BARTC[-1][0]+tcft[0], np.sqrt(BARTC[-1][1]**2+tcft[1]) ])
    #JARTCC.append( [JARTC[-1][0]+tcft[0], np.sqrt(JARTC[-1][1]**2+tcft[1]) ])


In [ ]:
JAR[:10]

In [ ]:
JAR2[:10]

In [ ]:
data_list = [JAR, GBAR, MBAR, GBARL,]
name_list = ['JAR','OBAR', 'MBAR','LOGBAR']

fig, ax = plt.subplots(len(data_list),1, figsize=(12,12), sharex=True, sharey=True)

for a, data, label in zip(ax, data_list, name_list):
    variance_plot(data, ax=a, parameter=F, z_score=1.64)
    print(f'{label} tot_err ={np.mean(np.abs((np.array(data)[:,1])))}')
    a.set_ylabel(label)
    
#ax[0].set_ylim(F+2,F-2)

In [ ]:
np.array(corr).mean(axis=0)

In [ ]:
datas = [BARTC, BARTCC, MBAR, JARTC, JARTCC, JAR]
labels = ['BAR_TC','BAR_TC - $\\log(P/R)$','BAR FULL','JAR_TC','JAR_TC - $\\log(P/R)$','JAR_FULL']

fig, ax = plt.subplots(2,3, figsize=(18,12), sharex=True, sharey='col')

Z = 1.64
for data, label, a in zip(datas, labels, ax.ravel() ):
    variance_plot(data, ax=a, parameter=F, z_score=Z)
    m=np.mean([d[0] for d in data])
    s=np.std([d[0] for d in data])
    a.axhspan(m-Z*s, m+Z*s, alpha=.3, color='g', label='sampling err')
    a.set_ylabel(label)


ax[0,1].set_ylim(F-3.5,F+3.5)
ax[0,2].set_ylim(F-3.5,F+3.5)

ax[0,0].set_ylim(F-3.5,F+3.5)


In [ ]:
datas = [BARTC, BARTCC, MBAR, JARTC, JARTCC, JAR]
labels = ['BAR_TC','BAR_TC - $\\log(P/R)$','BAR FULL','JAR_TC','JAR_TC - $\\log(P/R)$','JAR_FULL']

fig, ax = plt.subplots(2,3, figsize=(18,12), sharex=True, sharey='col')

Z = 1.64
for data, label, a in zip(datas, labels, ax.ravel() ):
    variance_plot(data, ax=a, parameter=F, z_score=Z)
    m=np.mean([d[0] for d in data])
    s=np.std([d[0] for d in data])
    a.axhspan(m-Z*s, m+Z*s, alpha=.3, color='g', label='sampling err')
    a.set_ylabel(label)


ax[0,1].set_ylim(F-3.5,F+3.5)
ax[0,2].set_ylim(F-3.5,F+3.5)

ax[0,0].set_ylim(F-3.5,F+3.5)


In [ ]:
#fig.savefig('tcftPoverR7p5.png')

In [ ]:
W_a

In [ ]:
W_b

In [ ]:
epsilon = .05
W_a = np.log(2*(1-epsilon))
W_b = np.log(2*epsilon)

F = epsilon*np.exp(W_a) + (1-epsilon)*np.exp(W_b)


In [ ]:
B=[]
C1 = []
C2 = []
J = []
BC1 = []
BC2 = []
tcc = []

In [ ]:


N=500

for i in range(N):
    
    print(f'{(i+1)/trials:.2f}', end='\r')
    n_samples= 10
    
    xF = np.random.uniform(0,1, n_samples)
    c1 = xF > epsilon
    c2 = xF <= epsilon
    
    xF[c1] = W_a
    xF[c2] = W_b
    
    xR = np.random.uniform(0,1, n_samples)
    
    rc1 = xR > .5
    rc2 = xR <= .5
    
    xR[rc1] = -W_a
    xR[rc2] = -W_b

    
    sums = [boo.sum() for boo in [c1,c2,rc1,rc2]]    
    if np.prod(sums)==0:
            continue
    
    B.append(bennet_delta_F(xF, xR))
    
    J.append(class_meta_f(xF))
    
    C1.append(class_meta_f(xF[c1], probability_bools=[c1, rc1]))
    
    BC1.append(bennet_delta_F(xF[c1], xR[rc1], probability_bools=[c1,rc1]))
    
    
    C2.append(class_meta_f(xF[c2], probability_bools=[c2, rc2]))

    BC2.append(bennet_delta_F(xF[c2], xR[rc2], probability_bools=[c2,rc2]))

    

In [ ]:
dat_list = [B, J, BC1, C1, BC2, C2]
names=['bar$(w, w^{R}$)','jar(w)',
       'bar$(w,w^{R}|C1) - \\log P(C1)+ \\log R(C1^{R})$',
       'jar$(w|C1) - \\log P(C1)+ \\log R(C1^{R})$', 
       'bar$(w,w^{R}|C2) - \\log P(C2)+ \\log R(C2^{R})$',
       'jar$(x|C2) - \\log P(C2)+ \\log R(C2^{R})$']

for i in [5,4]:
    dat_list.remove(dat_list[i])
    names.remove(names[i])
fig, ax = plt.subplots(1,len(dat_list), figsize=(23,6), sharey=True)
for d, a, l in zip(dat_list, ax.ravel(), names):
    variance_plot(d, ax=a, parameter=F, z_score=1.64)
    a.set_ylabel(l)

In [ ]:
#fig.savefig('bar_vs_jar_n_10_trials_120_epsilon_p1.png')

In [ ]:
B[-1]

In [ ]:
tcft_correction(c1,rc1)

In [ ]:
c

In [ ]:
J[-1]

In [ ]:
BC1[-1]

In [ ]:
C1[-1]

In [ ]:
bennet_delta_F(xF[c2], xR[rc2])

In [ ]:
class_meta_f(xF[c2])

In [ ]:
xR[rc2].shape

In [ ]:
from free_energy import tcft_correction, get_proportion_error

ests = []
ests_num = []

p = .2
pR = .1

for i in range(500):
    P = np.random.uniform(0,1,30) < p
    R = np.random.uniform(0,1, 30) < pR
    
    PR, VPR = tcft_correction(P,R)
    if PR != np.inf:
        ests.append([PR, np.sqrt(VPR)])

    

In [ ]:
fig, ax = plt.subplots()

variance_plot(ests, ax=ax, parameter=-np.log(p/pR), z_score=1.96)